# Problem set 2: part one

Total points: 20

Fraction of problem set two points: 25\%

**Concepts**: here, you're going to start using the SIP H-2A employer jobs and debarment data to practice exact matching,  regex, and fuzzy matching

In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage
! pip install recordlinkage

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Regex and exact matching (8 points)



# 1.1 Load data on debarments and job postings (0 points)

Load the following datasets stored in `pset2_part1_inputdata`
    
- Historical H2A debarments (debar.csv); call this `debar`
- Q1 2021 H2A job postings (jobs.csv); call this `jobs`


In [2]:
debar = pd.read_csv("debar.csv")
jobs = pd.read_csv("jobs.csv")



### 1.2 Try exact merge on business name  (2 points)

- Use the `EMPLOYER_NAME` field of jobs
- Use the `Name` field of debar

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are matches, print the name in each dataset, date range of debarment (`Start date` and `End date` in `debar`) and location from each data (`City, State` in `debar` and `EMPLOYER_CITY` and `EMPLOYER_STATE` in `jobs`)

**Resources**:
    - Slides 17-19 here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/qss20_s21_class4.pdf 
    - Code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/03_merging_session1.ipynb 


In [3]:
##A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

merge_name = pd.merge(jobs, debar ,
how = "inner",
left_on = "EMPLOYER_NAME",
right_on = "Name")

##B
matched_debar = debar[debar['Name'].isin(merge_name.Name)]               
matched_debar = matched_debar[['Name','City, State', "Start date", "End date"]]
matched_job = jobs[jobs['EMPLOYER_NAME'].isin(merge_name.EMPLOYER_NAME)]               
matched_job = matched_job[['EMPLOYER_NAME','EMPLOYER_CITY', "EMPLOYER_STATE"]]                                                                     

#Match in debar
matched_debar
#Matches in jobs
matched_job
#Match combined
merge_name=merge_name[['Name', 'EMPLOYER_NAME', "Start date", "End date", "City, State", "EMPLOYER_CITY", "EMPLOYER_STATE"]]
merge_name






,Name,"City, State",Start date,End date
63,Rafael Barajas,"Sebring, Florida",9/23/2016,9/22/2017


,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE
791,Rafael Barajas,Port St. Lucie,FL


,Name,EMPLOYER_NAME,Start date,End date,"City, State",EMPLOYER_CITY,EMPLOYER_STATE
0,Rafael Barajas,Rafael Barajas,9/23/2016,9/22/2017,"Sebring, Florida",Port St. Lucie,FL


## 1.3 Targeted regex

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the name fields.


 
A. Convert the names `EMPLOYER_NAME` and `Name` to uppercase using list comprehension rather than df.varname.str.upper()

B. Print a random sample of 15 values of each result

C. Assign them back to the original data, writing over the original columns

**Resources**:
    - `new_colnames` line of code here for example of list comprehension: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/04_basicregex_formerging.ipynb
    - Sampling from a list without replacement using the `random` module: https://note.nkmk.me/en/python-random-choice-sample-choices/ 
    

In [4]:

## A. Convert the names `EMPLOYER_NAME` and `Name` to uppercase using list comprehension 
## rather than df.varname.str.upper()
upper_employer = [x.upper() for x in jobs['EMPLOYER_NAME']]
upper_name = [x.upper() for x in debar['Name']]


## B. Print a random sample of 15 values of each result
random.sample(upper_employer,15)
random.sample(upper_name,15)

## C.Assign them back to the original data, writing over the original columns
jobs.EMPLOYER_NAME=upper_employer
debar.Name=upper_name


['J & A PHILLIPS RANCH',
 'LONE WOLF MANAGEMENT, INC.',
 'BADILLO BROTHERS, INC.',
 'DORE FARMS',
 'RONALD J MOSS',
 'TM HARVESTING, INC.',
 'MILLER BUFFALO RANCH',
 'BELA FLOR NURSERIES, INC.',
 'CHRISTMAS FLYING SERVICE, LLC',
 'ROARING FORK SHEEP CO.',
 'BYRON P. ZAUNBRECHER',
 'GOEBEL FARMS, LLC',
 'MARVIN MONCEAUX',
 'WINBAK FARM, LLC',
 'VESPER RANCH']

['MAPLE RIDGE CUSTOM SERVICES, LLC',
 'TURNER FARMS',
 'DAVID JACKSON',
 'JOHN R. COOK',
 'PROMAX INC.',
 'RAFAEL BARAJAS',
 'AUTUMN HILL ORCHARD',
 'DOVE CREEK FARMS',
 'LESLIE RENEE DREW',
 'GERONIMO SHEEP CO.',
 'GLENDA HARROD FARMS',
 'LEONARD SMITH FARMS',
 'GREAT PLAINS FLUID SERVICE, INC.',
 'DEAL FAMILY FARM',
 'CIRA CORTEZ LOPEZ']

### 1.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a . but sometimes not

A. For each dataset, write a regex pattern to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example and print the result. The first positive example should return `CISCO PRODUCE INC`; the second positive example should return `AVOYELLES HONEY CO, LLC` the negative example should return `E.V. RANCH LLP` (so leave the dots between E.V. in)

C. Execute on (1) Name in `debar` (can just write over the col) and (3) EMPLOYER_NAME  in `jobs`, making sure to use the uppercase versions of the variables

**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content

**Resources**:
    - Regex slides are 22-40: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/qss20_s21_class4.pdf 
    - Regex code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/04_basicregex_formerging.ipynb

In [5]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [6]:
## A & B


pattern01 = r"([INC|CO|LLC])\."
pattern02=r'\1'
re.sub(pattern01, pattern02, pos_example_1)
re.sub(pattern01, pattern02, pos_example_2)
re.sub(pattern01, pattern02, neg_example)

## C

    
debar.Name = [re.sub(pattern01, pattern02, name) 
                   for name in debar.Name]

debar.Name.sample(15)

jobs.EMPLOYER_NAME = [re.sub(pattern01, pattern02, name) 
                   for name in jobs.EMPLOYER_NAME]


jobs.EMPLOYER_NAME.sample(15)



'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

49               CITY PINESTRAW AND HARVESTING
104           LOV RANCH CO, LLLP DBA LOV RANCH
1                       STAHLMAN APIARIES, INC
51     OLD TREE FARMS/VERPAALEN CUSTOM SERVICE
22                        ALTERIC JEAN-CHARLES
7                          AUTUMN HILL ORCHARD
105                                J & L FARMS
27                      YOLANDA CHAVEZ FARMING
55                        MACKY AND BRAD FARMS
38                                 LESLIE COOK
94                                69 FARMS LLC
16                                 MARK DUNCAN
58                                   F&W FARMS
77                       MYRKA MIREYA CARDENAS
53                               AGECY II, LLC
Name: Name, dtype: object

1939              CANDELARIO SEPULVEDA JUAREZ
38                     HUMPHREY COKER SEED CO
777                     DAVID OR JANINE FOLEY
2260                            CK FARMS, LLC
523                  WALTER P RAWL & SONS INC
1423                  LANCE HABETZ FARMS, LLC
2055    TEXAS MARICULTURE - CARANCAHUA BAY LP
205              MCNEILL LABOR MANAGEMENT INC
1106                   COIGNARD BROTHERS, LLC
1294                      KYLE FONTENOT FARM 
2101                       CHAD MARTIN THEVIS
1847                G & M BROUSSARD FARM, LLC
1340                          FORREST ARTHUR 
2060                           BAILEY COMPANY
2460                         CARONA FARMS LLC
Name: EMPLOYER_NAME, dtype: object

### 1.3.3 OPTIONAL extra credit - regex to separate companies from individuals (2 points)

You notice some employers in `debar` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned version of the Name in `debar`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string-- so in above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the and)
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
C. Iterate over the cleaned `Name` column in debar and execute the regex. Use it to create two new columns in debar:
    - A column for company (full Name string if left as is; pattern before COMPANY if one extracted)
    - A column for individual (full Name string if left as is; pattern before INDIVIDUAL if one extracted)
    
D. Print those columns for the rows containing the negative example and positive example

**Hint**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

For step C, you probably want to use a loop or function that uses if else to do different things if a match is found or not


In [7]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

In [8]:

#A&B
pattern03=r"^(.*)([(]COMPANY[)])\s*(AND)\s*(.*)([(]INDIVIDUAL[)])(\*?)"
re.findall(pattern03,pos_example)
re.findall(pattern03,neg_example)
pattern04=r"^(.*)\s*([(]INDIVIDUAL[)])"

debar.Name.astype("string")


#C
added = []
for name in (debar.Name):
    name_check=re.findall(pattern03,name)
    name_check_additional=re.findall(pattern04,name)
    if(len(name_check))==0: 
        if (len(name_check_additional)==0):
            sth=debar[debar["Name"]==name].index.values
            sth=sth.astype("int")
            debar.loc[sth, 'company'] = name
            debar.loc[sth, 'individual'] = name
            data=debar.loc[debar['Name'] == name]
            added.append(data) 
    if(len(name_check))>0: 
        name_check=name_check[0]
        sth=debar[debar["Name"]==name].index.values
        sth=sth.astype("int")
        debar.loc[sth, 'company'] = name_check[0]
        debar.loc[sth, 'individual'] = name_check[3]
        data=debar.loc[debar['Name'] == name]
        added.append(data)
    if (len(name_check_additional)>0):
        if (len(name_check))==0:
            name_check_additional=name_check_additional[0]
            sth=debar[debar["Name"]==name].index.values
            sth=sth.astype("int")
            debar.loc[sth, 'company'] = name
            debar.loc[sth, 'individual'] = name_check_additional[0]
            data=debar.loc[debar['Name'] == name]
            added.append(data)
        
final = pd.concat(added)
final=final.drop_duplicates()    
final
#D
pos=final[(final["Name"] == "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*")]
neg=final[(final["Name"] == "CISCO PRODUCE INC")]
pos
neg       


[('COUNTY FAIR FARM ',
  '(COMPANY)',
  'AND',
  'ANDREW WILLIAMSON ',
  '(INDIVIDUAL)',
  '*')]

[]

0                                   J&J HARVESTING
1                           STAHLMAN APIARIES, INC
2                                    TRUST NURSERY
3                             ANTON FERTILIZER INC
4                  GREAT PLAINS FLUID SERVICE, INC
                          ...                     
109                               DOVE CREEK FARMS
110                                  JESUS LEDESMA
111                                   TURNER FARMS
112    B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)
113                                    DELIA ROJAS
Name: Name, Length: 114, dtype: string

,Name,"City, State",Violation,Duration,Start date,End date,company,individual
0,J&J HARVESTING,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016,J&J HARVESTING,J&J HARVESTING
1,"STAHLMAN APIARIES, INC","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016,"STAHLMAN APIARIES, INC","STAHLMAN APIARIES, INC"
2,TRUST NURSERY,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,TRUST NURSERY,TRUST NURSERY
3,ANTON FERTILIZER INC,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,ANTON FERTILIZER INC,ANTON FERTILIZER INC
4,"GREAT PLAINS FLUID SERVICE, INC","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,"GREAT PLAINS FLUID SERVICE, INC","GREAT PLAINS FLUID SERVICE, INC"
...,...,...,...,...,...,...,...,...
107,WALKER PLACE,"Danville, IL",Failure to comply with the employer's obligati...,2 months,11/19/2019,1/26/2020,WALKER PLACE,WALKER PLACE
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,COUNTY FAIR FARM,ANDREW WILLIAMSON
110,JESUS LEDESMA,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,JESUS LEDESMA,JESUS LEDESMA
112,B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),"Collins, Georgia",WHD Debarment,3 years,4/9/17,4/9/20,B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ


,Name,"City, State",Violation,Duration,Start date,End date,company,individual
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,COUNTY FAIR FARM,ANDREW WILLIAMSON


,Name,"City, State",Violation,Duration,Start date,End date,company,individual
19,CISCO PRODUCE INC,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,CISCO PRODUCE INC,CISCO PRODUCE INC
56,CISCO PRODUCE INC,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,CISCO PRODUCE INC,CISCO PRODUCE INC


### 1.4.1 Preprocessing location (2 points)

You want to block on state but notice that states in `debar` have a mix of two digit codes and full state names (e.g., GA versus Georgia) while states in `jobs` are all two-digit codes

A. Run the code below to load the `states` crosswalk. Use that crosswalk to create a field in `debar` that has the two-digit state abbreviation for all locations (hint: you may need to first split the string on the ", " or use str.replace to extract the state)

B. Use an `assert` statement to check that all the states in `debar` are two-digits after the cleaning

**Notes**: you can filter out states that are NaN

In [9]:
states = pd.read_csv("states_pset2.csv")
debar['state_pre'] = debar["City, State"].astype('string').str.replace("^(.*?)\,", "", regex = True).str.strip()
debar=debar[~debar.state_pre.isnull()]
debar["state_not2"]= np.where(debar.state_pre.str.len()>2, True, False)

state_uncleaned = debar.loc[debar['state_not2'] ==True]

state_uncleaned = state_uncleaned.rename(columns={'state_pre': 'state_pre2'})


states['Code'] = states["Code"].astype('string')
states['Description'] = states["Description"].astype('string')

merge_state = pd.merge(state_uncleaned, states,
how = "inner",
left_on = "state_pre2",
right_on = "Description")

state_cleaned = debar.loc[debar['state_not2'] ==False]
merge_state = merge_state.drop(columns=['state_pre2', 'state_not2', "Description"])
merge_state = merge_state.rename(columns={'Code': 'state_pre'})
state_cleaned = state_cleaned.drop(columns=["state_not2"])
debar_new = pd.concat([merge_state, state_cleaned])
debar_new = debar_new.rename(columns={'state_pre': 'state'})
assert all(debar_new.state.str.len()==2), "All the states in debar are not two-digits after the cleaning"


### 1.4.2 step by step fuzzy matching (4 points)



A. Write fuzzy matching code (don't yet put inside a user-defined function) that:

- Blocks on two-digit state code
- Finds matches based on similarity between the employer name (`Name`) in `debar` (uppercase and cleaned) and `EMPLOYER_NAME` in `jobs` (uppercase and cleaned). You can choose which distance metric and threshold to use (feel free to set a threshold low enough to get some matches even if that leads to some false positives).

For the steps after compute, just take any match with non-zero value rather than using a classifier (so skip the k-means or e-m step in the example code)

B. Print the matches and comment on examples of ones that seem like true positives and ones that seem like false matches

**Resources**:

- Solutions code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/05_merging_session2_activitysolutions.ipynb 
- Example code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/05_merging_session2_codeexample.ipynb


**Hint**: you may need to deduplicate multiple records in the datasets for the recordlinkage package to work. See drop_duplicates within pandas and subset command

In [10]:
## Rename columns that are going to be matched

debar_new = debar_new.rename(columns={'Name': 'name_4match'})
jobs = jobs.rename(columns={'EMPLOYER_NAME': 'name_4match'})

## Rename two digit state code column in jobs dataset
jobs = jobs.rename(columns={'EMPLOYER_STATE': 'state'})

# # deduplicate
debar_new=debar_new.drop_duplicates(subset="name_4match").reset_index()

jobs=jobs.drop_duplicates(subset="name_4match").reset_index()

# ## initialize indexer
matcher = recordlinkage.Index()
matcher.block("state")

candidate_links = matcher.index(debar_new, jobs)
candidate_links



<Index>

MultiIndex([( 0,   30),
            ( 0,   74),
            ( 0,  109),
            ( 0,  122),
            ( 0,  134),
            ( 0,  142),
            ( 0,  190),
            ( 0,  229),
            ( 0,  258),
            ( 0,  297),
            ...
            (94, 1064),
            (94, 1293),
            (94, 1426),
            (94, 1796),
            (94, 2061),
            (95,  256),
            (95,  662),
            (95,  691),
            (95, 1049),
            (95, 2051)],
           length=9088)

In [11]:
compare = recordlinkage.Compare()

thres = 0.85
compare.string('name_4match', 'name_4match', method='jarowinkler', threshold=thres)

compare_vectors = compare.compute(candidate_links, debar_new, jobs)
print(type(compare_vectors))

compare_vectors

<Compare>

<class 'pandas.core.frame.DataFrame'>


0
0  30    0.0
   74    0.0
   109   0.0
   122   0.0
   134   0.0
...      ...
95 256   0.0
   662   0.0
   691   0.0
   1049  0.0
   2051  0.0

[9088 rows x 1 columns]

In [12]:
## For the steps after compute, just take any match with non-zero value rather than using a classifier 
## so skip the k-means or e-m step in the example code
compare_vectors.columns = ["name"]
df_resultsfromcompute=compare_vectors[(compare_vectors.name == 1)]
df_resultsfromcompute=df_resultsfromcompute.index


indices_debar = [x[0] for x in df_resultsfromcompute]
indices_jobs = [x[1] for x in df_resultsfromcompute]

df_matchpairs = pd.DataFrame({'debar_indices': indices_debar,
                'jobs_indices': indices_jobs})

debar_new['index_4merge'] = debar_new.index
jobs['index_4merge'] = jobs.index


df_matchpairs_debar = pd.merge(df_matchpairs,
                            debar_new[['name_4match','state',"index_4merge"]],
                            how = "left",
                            left_on = "debar_indices",
                            right_on = "index_4merge")

df_matchpairs_both = pd.merge(df_matchpairs_debar,
                              jobs[['name_4match','state',"index_4merge"]],
                              how = "left",
                              left_on = "jobs_indices",
                              right_on = "index_4merge",
                              suffixes= ["_debar", "_jobs"])
df_matchpairs_both




,debar_indices,jobs_indices,name_4match_debar,state_debar,index_4merge_debar,name_4match_jobs,state_jobs,index_4merge_jobs
0,44,358,F&W FARMS,KS,44,D&L FARMS,KS,358
1,44,1701,F&W FARMS,KS,44,L&E FARMS,KS,1701
2,3,1674,TRIANGLE J FARMS,TX,3,TRIANGLE TURF COMPANY INC,TX,1674
3,7,637,RAFAEL BARAJAS,FL,7,RAFAEL BARAJAS,FL,637
4,18,526,GERMAN FARMS,ND,18,TESSMAN FARMS,ND,526
5,80,1150,OLSON FARMS,ND,80,NELSON FARMS,ND,1150
6,24,339,"AVOYELLES HONEY CO, LLC",LA,24,AVOYELLES CRAWFISH & SEAFOOD LLC,LA,339
7,92,1499,J & L FARMS,SD,92,S & S FARMS,SD,1499
8,46,894,SLASH E.V. RANCH LLP,CO,46,SLASH EV RANCH,CO,894
9,91,835,"LOV RANCH CO, LLLP DBA LOV RANCH",CO,91,LOV RANCH,CO,835


In [13]:
## True positive matches:
## TRIANGLE J FARMS/TRIANGLE TURF COMPANY INC>> both have traingle in it as the main name
## RAFAEL BARAJAS/RAFAEL BARAJAS>> (perfect match)
## SLASH E.V. RANCH LLP/SLASH EV RANCH>>nearly perfect match despite a slight punctuation difference
## LOV RANCH CO, LLLP DBA LOV RANCH/LOV RANCH>>nearly perfect match despite a slight punctuation difference and business type specification
## AVOYELLES HONEY C, LLC/AVOYELLES CRAWFISH & SEAFOOD LLC>>both have AVOYELLES in it as the main name
## DIXIE BELL #2/DIXIE BELLE, IN>>nearly perfect match despite a slight business type label difference

## False positive matches:
## F&W FARMS/D&L FARMS >>quite different company names' acromym, so probably not the real match
## F&W FARMS/L&E FARMS >>quite different company names' acromym, so probably not the real match
## GERMAN FARMS/TESSMAN FARMS >>it might match on "man" but these two farms have different names
## OLSON FARMS/NELSON FARMS >>it might match on "son" but these two farms have different names
## J & L FARMS/S & S FARMS >>quite different company names' acromym, so probably not the real match


### 1.4.3 write a fuzzy matching function (6 points)

You want to see how the matches change if you add the town and not only state as a field and also want to automate the process of matching a bit to try different distance thresholds.

A. Extract the City from the `City, State` column of debar

B. Convert that new `city` column to uppercase and convert the `EMPLOYER_CITY` column in jobs to uppercase

C. Write a function surrounding the code in `recordlinkage` (so you don't need to recode the package from scratch) that (1) takes in each dataset, (2) blocks on two-digit state, and (3) fuzzy matches on employer name and employer city

D. Execute the function with a couple different string distance thresholds and print the results of each

5 out of 6 points: function takes arguments for input datasets, varname to block on, two varnames to fuzzy match on, string distance function, and string distance threshold
    
6 out of 6: above but function is also general enough that it takes a variable # of strings to match on--- so should work if you either execute just using employer name or also work if you execute using employer name and employer city

In [14]:
debar_new["city"]=debar_new["City, State"].astype('string').str.replace("(,).*" ,"", regex = True)
debar_new["city"]=[x.upper() for x in debar_new['city']]
jobs["city"]=[x.upper() for x in jobs['EMPLOYER_CITY']]
jobs["city"].astype("string").str.strip()




0             PORTLAND
1          MCMINNVILLE
2           LIVINGSTON
3             BROCKTON
4         WINTER HAVEN
             ...      
2129           OAKLAND
2130            ELKTON
2131    AMERICAN FALLS
2132          BELLEVUE
2133        ORWIGSBURG
Name: city, Length: 2134, dtype: string

In [15]:
def fuzzy_matching(data1,data2,sd_func,thres,bvarname,**kwargs):
    lst = {}
    for key, value in kwargs.items():
        lst[key] = value
    matcher = recordlinkage.Index()
    matcher.block(bvarname)
    candidate_links = matcher.index(data1, data2)
    
    compare = recordlinkage.Compare()
    for key in lst:
        compare.string(lst[key], lst[key], method=sd_func, threshold=thres)
    compare_vectors = compare.compute(candidate_links, data1, data2)

    kmeans = recordlinkage.KMeansClassifier()
    kmeans_results = kmeans.fit_predict(compare_vectors)

    indices_debar = [x[0] for x in kmeans_results]
    indices_jobs = [x[1] for x in kmeans_results]

    df_matchpairs = pd.DataFrame({'debar_indices': indices_debar,
                    'jobs_indices': indices_jobs})

    debar_new['index_4merge'] = debar_new.index
    jobs['index_4merge'] = jobs.index
    
    values=lst.values()
    comparelist = list(values)
    comparelist.append(bvarname)
    comparelist.append("index_4merge")
    df_matchpairs_debar = pd.merge(df_matchpairs, data1[comparelist], how = "left",
                                left_on = "debar_indices", right_on = "index_4merge")
    df_matchpairs_both = pd.merge(df_matchpairs_debar, data2[comparelist], how = "left",
                                      left_on = "jobs_indices", right_on = "index_4merge",
                                      suffixes= ["_debar", "_jobs"])
    return df_matchpairs_both
print("0.7 threshold two mvarname")
fuzzy_matching(data1=debar_new, data2=jobs,sd_func='jarowinkler',thres=0.7,bvarname='state',mvarname1='city',mvarname2='name_4match')
print("0.8 threshold two mvarname")
fuzzy_matching(data1=debar_new, data2=jobs,sd_func='jarowinkler',thres=0.8,bvarname='state',mvarname1='city',mvarname2='name_4match')
print("0.9 threshold two mvarname")
fuzzy_matching(data1=debar_new, data2=jobs,sd_func='jarowinkler',thres=0.9,bvarname='state',mvarname1='city',mvarname2='name_4match')
print("0.9 threshold one mvarname")
fuzzy_matching(data1=debar_new, data2=jobs,sd_func='jarowinkler',thres=0.9,bvarname='state',mvarname2='name_4match')


0.7 threshold two mvarname


,debar_indices,jobs_indices,city_debar,name_4match_debar,state_debar,index_4merge_debar,city_jobs,name_4match_jobs,state_jobs,index_4merge_jobs
0,85,103,LYONS,DAVID C MARTINEZ,GA,85,LYONS,JOSE H. MARTINEZ,GA,103
1,18,1995,FULLERTON,GERMAN FARMS,ND,18,FLAXTON,SORUM FARMS,ND,1995
2,24,339,MOREAUVILLE,"AVOYELLES HONEY CO, LLC",LA,24,MOREAUVILLE,AVOYELLES CRAWFISH & SEAFOOD LLC,LA,339
3,46,894,RIFLE,SLASH E.V. RANCH LLP,CO,46,RIFLE,SLASH EV RANCH,CO,894
4,69,835,RIFLE,E.V. RANCH LLP,CO,69,RIFLE,LOV RANCH,CO,835
5,91,835,RIFLE,"LOV RANCH CO, LLLP DBA LOV RANCH",CO,91,RIFLE,LOV RANCH,CO,835


0.8 threshold two mvarname


,debar_indices,jobs_indices,city_debar,name_4match_debar,state_debar,index_4merge_debar,city_jobs,name_4match_jobs,state_jobs,index_4merge_jobs
0,24,339,MOREAUVILLE,"AVOYELLES HONEY CO, LLC",LA,24,MOREAUVILLE,AVOYELLES CRAWFISH & SEAFOOD LLC,LA,339
1,46,894,RIFLE,SLASH E.V. RANCH LLP,CO,46,RIFLE,SLASH EV RANCH,CO,894
2,91,835,RIFLE,"LOV RANCH CO, LLLP DBA LOV RANCH",CO,91,RIFLE,LOV RANCH,CO,835


0.9 threshold two mvarname


,debar_indices,jobs_indices,city_debar,name_4match_debar,state_debar,index_4merge_debar,city_jobs,name_4match_jobs,state_jobs,index_4merge_jobs
0,46,894,RIFLE,SLASH E.V. RANCH LLP,CO,46,RIFLE,SLASH EV RANCH,CO,894


0.9 threshold one mvarname


,debar_indices,jobs_indices,name_4match_debar,state_debar,index_4merge_debar,name_4match_jobs,state_jobs,index_4merge_jobs
0,7,637,RAFAEL BARAJAS,FL,7,RAFAEL BARAJAS,FL,637
1,46,894,SLASH E.V. RANCH LLP,CO,46,SLASH EV RANCH,CO,894
2,73,1007,DIXIE BELL #2,SC,73,"DIXIE BELLE, INC",SC,1007
